## Exploring the Walmart Amazon dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = ''

#### Install the autolabel library

In [2]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [3]:
from autolabel import get_data

get_data('walmart_amazon')

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100% [........................................................] 103732 / 103732

100% [........................................................] 929245 / 929245

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

In [2]:
import json

from autolabel import LabelingAgent

In [3]:
# load the config
with open('config_walmart_amazon.json', 'r') as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `entity_matching` (since it's an entity matching task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.labels`: `['duplicate', 'not duplicate']` (the full list of labels to choose from)
* `prompt.task_guidelines`: `'You are an expert at identifying duplicate products from online product catalogs...` (how we describe the task to the LLM)
* `prompt.few_shot_num`: 10 (how many labeled examples to provide to the LLM)

In [4]:
config

{'task_name': 'ProductCatalogEntityMatch',
 'task_type': 'entity_matching',
 'dataset': {'label_column': 'label', 'delimiter': ','},
 'model': {'provider': 'llama', 'name': '/workspace/hf-trained-nonchat'},
 'embedding': {'provider': 'openai'},
 'prompt': {'task_guidelines': 'Your job is to correctly label the following into one of the following categories {labels}',
  'output_guidelines': '\\n',
  'labels': ['duplicate', 'not duplicate'],
  'example_template': 'Title of entity1: {Title_entity1}; category of entity1: {Category_entity1}; brand of entity1: {Brand_entity1}; model number of entity1: {ModelNo_entity1}; price of entity1: {Price_entity1}\nTitle of entity2: {Title_entity2}; category of entity2: {Category_entity2}; brand of entity2: {Brand_entity2}; model number of entity2: {ModelNo_entity2}; price of entity2: {Price_entity2}\nDuplicate or not: {label}',
  'few_shot_examples': [{'Title_entity1': 'lexmark extra high yield return pgm print cartridge - magenta',
    'Category_enti

In [5]:
# create an agent for labeling
agent = LabelingAgent(config=config)

INFO 08-25 21:43:45 llm_engine.py:70] Initializing an LLM engine with config: model='/workspace/hf-trained-nonchat', tokenizer='/workspace/hf-trained-nonchat', tokenizer_mode=auto, trust_remote_code=False, dtype=torch.float16, use_dummy_weights=False, download_dir=None, use_np_weights=False, tensor_parallel_size=1, seed=0)


2023-08-25 21:43:46 torch.distributed.distributed_c10d INFO: Added key: store_based_barrier_key:1 to store for rank: 0
2023-08-25 21:43:46 torch.distributed.distributed_c10d INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 1 nodes.
2023-08-25 21:43:46 torch.distributed.distributed_c10d INFO: Added key: store_based_barrier_key:2 to store for rank: 0
2023-08-25 21:43:46 torch.distributed.distributed_c10d INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:2 with 1 nodes.
2023-08-25 21:43:46 torch.distributed.distributed_c10d INFO: Added key: store_based_barrier_key:3 to store for rank: 0
2023-08-25 21:43:46 torch.distributed.distributed_c10d INFO: Rank 0: Completed store-based barrier for key:store_based_barrier_key:3 with 1 nodes.
2023-08-25 21:43:46 torch.distributed.distributed_c10d INFO: Added key: store_based_barrier_key:4 to store for rank: 0
2023-08-25 21:43:46 torch.distributed.distributed_c10d INFO: Rank 0: Completed stor

INFO 08-25 21:44:53 llm_engine.py:196] # GPU blocks: 1463, # CPU blocks: 327


In [6]:
from autolabel import AutolabelDataset
ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬──────┐
│ Total Estimated Cost     │ $0.0 │
│ Number of Examples       │ 1800 │
│ Average cost per example │ $0.0 │
└──────────────────────────┴──────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

<s>[INST] <<SYS>>
    Your job is to correctly label the following into one of the following categories duplicate
not duplicate\n
Title of entity1: lexmark extra high yield return pgm print cartridge - magenta; category of entity1: printers; 
brand of entity1: lexmark; model number of entity1: c782u1mg; price of entity1: 214.88
Title of entity2: lexmark 18c1428 return program print cartridge black; category of entity2: inkjet printer ink; 
brand of entity2: lexmark; model number of entity2: 18c1428; price of entity2: 19.97
Duplicate or not: not duplicate

Title of entity1: edge tech proshot 4gb sdhc class 6 memory card; category of entity1: usb drives; brand of 
entity1: edge tech; model number of entity1: pe209780; price of entity1: 10.88
Title of entity2: 4gb edge proshot sdhc memory card class6; category of entity2: computers accessories; brand of 
entity2: edge; model number of entity2: nan; price of entity2: 17.83
Duplicate or not: duplicate
    <<SYS>>
    Title of entity1: sony 16gb class 4 sd memory card; category of entity1: usb drives; brand of entity1: sony; 
model number of entity1: sf16n4/tqp; price of entity1: 0.0
Title of entity2: pny 4gb class 4 navy sd card; category of entity2: car audio video; brand of entity2: pny; model 
number of entity2: p-sdhc4g4-ef / navy; price of entity2: 11.18
Duplicate or not: [/INST]

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [8]:
ds = agent.run(ds, max_items=1000)

Output()

classification_report:
               precision    recall  f1-score   support

    duplicate       0.00      0.00      0.00         7
not duplicate       0.93      0.99      0.96        93

     accuracy                           0.92       100
    macro avg       0.46      0.49      0.48       100
 weighted avg       0.86      0.92      0.89       100

Actual Cost: 0.0

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.92     │ 100     │ 1.0             │
└──────────┴─────────┴─────────────────┘

We are at 97% accuracy when labeling the first 100 examples. Let's see if we can use confidence scores to improve accuracy further by removing the less confident examples from our labeled set.

### Compute confidence scores


In [12]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ['REFUEL_API_KEY'] = 'xxxxxxxxxxxxxxxxx'

In [13]:
config["model"]["compute_confidence"] = True

In [14]:
agent = LabelingAgent(config=config)

In [15]:
from autolabel import AutolabelDataset
ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $5.1813 │
│ Number of Examples       │ 1800    │
│ Average cost per example │ $0.0029 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at identifying duplicate products from online product catalogs.
Your job is to tell if the two given entities are duplicates or not duplicate. Your answer must be from one of the following options:
duplicate
not duplicate

You will return the answer with one element: "the correct option"


Some examples with their output answers are provided below:

Title of entity1: lexmark extra high yield return pgm print cartridge - magenta; category of entity1: printers; brand of entity1: lexmark; model number of entity1: c782u1mg; price of entity1: 214.88
Title of entity2: lexmark 18c1428 return program print cartridge black; category of entity2: inkjet printer ink; brand of entity2: lexmark; model number of entity2: 18c1428; price of entity2: 19.97
Duplicate or not: not duplicate

Title of entity1: edge tech proshot 4gb sdhc class 6 memory card; category of entity1: usb drives; brand of entity1: edge tech; model number of entity1: pe209780; price of entity1: 10.88
Title of enti

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [16]:
ds = agent.run(ds, max_items=100)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Metric: auroc: 0.9725
Actual Cost: 0.0043


┏━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 100     │ -inf      │ 0.97     │ 1.0             │
│ 1       │ 0.9963    │ 1.0      │ 0.01            │
│ 91      │ 0.71      │ 1.0      │ 0.91            │
│ 92      │ 0.6827    │ 0.9891   │ 0.92            │
│ 96      │ 0.6185    │ 0.9896   │ 0.96            │
│ 97      │ 0.605     │ 0.9794   │ 0.97            │
│ 99      │ 0.5024    │ 0.9798   │ 0.99            │
│ 100     │ 0.4291    │ 0.97     │ 1.0             │
└─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

Looking at the table above, we can see that if we set the confidence threshold at `0.71`, we are able to label at 100% accuracy and getting a completion rate of 91%. This means, we would ignore all the data points where confidence score is less than `0.71` (which would end up being around 9% of all samples). This would, however, guarantee a very high quality labeled dataset for us. 